# OpenRouteService python bindings

https://github.com/GIScience/openrouteservice-py

In [1]:
import openrouteservice as ors
import folium
import json
import random
from pprint import pprint

url_ors = 'http://192.168.1.7:8080/ors'  #istanze locale ROUTING ENGINE 

url_vroom = 'http://vroom:8081'  #instanza loale VROOM

# DATI json posizione jobs (Bidoni)
pointsA = json.load(open('./data/mezzolombardo.geojson'))['features'] #4 points
pointsB = json.load(open('./data/mezzocorona.geojson'))['features'] #6 points

center = [46.2147, 11.1065]
zoom = 15

# POSIZIONI PARTENZA DEI VEICOLI (Furgoni)
point_start = [46.2170,11.1066]
# POSIZIONE ARRIVO DEI VEICOLI
point_end = [point_start[0], point_start[1]]


# OpenRouteService Simple TSP Routing Problem ( without VROOM )

example of one vehicle for two groups of bins all full, without considering the capacity of vehicle.

- [API Directions](https://openrouteservice.org/dev/#/api-docs/v2/directions/{profile}/post)
- [Preferences examples](https://github.com/GIScience/openrouteservice-docs#examples)
- [more examples](https://github.com/GIScience/openrouteservice-py/blob/master/examples/basic_example.ipynb)

In [2]:
M = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)
    
client = ors.Client(base_url=url_ors, key='') #LIBRERIA ORS STANDARD

ccA = [p['geometry']['coordinates'] for p in pointsA]
ccB = [p['geometry']['coordinates'] for p in pointsB]

#include partenza e arrivo
dirccA = [list(reversed(point_start))] + ccA + [list(reversed(point_end))]
dirccB = [list(reversed(point_end))] + ccB + [list(reversed(point_end))]

# TSP 1
routeA = client.directions(
    coordinates = dirccA,
    profile = 'driving-hgv',
    format = 'geojson',
    preference = 'fastest',#shortest
    #restrictions =  ... width,height(in metri), weight(in tonnellate)
    validate = False
)

# TSP 1
routeB = client.directions(
    coordinates = dirccB,
    profile = 'driving-car',
    format = 'geojson',
    validate = False
)

polyA = [list(reversed(coord)) for coord in routeA['features'][0]['geometry']['coordinates']]
polyB = [list(reversed(coord)) for coord in routeB['features'][0]['geometry']['coordinates']]

folium.PolyLine(polyA, color='red').add_to(M)
folium.PolyLine(polyB, color='blue').add_to(M)

for c in (ccA + ccB):
    loc = list(reversed(c))
    folium.Marker(loc, icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(M)

folium.Marker(point_start, icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(M)
    
M

## VRP Vehicle Routing Problem  (using VROOM)

Optimize job scheduling for multiple vehicles.
The next example makes more use of the power of the ORS optimization endpoint(using Vroom)
- [API Optimization](https://openrouteservice.org/dev/#/api-docs/optimization/post)
- [API VROOM](https://github.com/VROOM-Project/vroom/blob/master/docs/API.md)

**Definition of 2 vehicles each having capacity 5 and bins having contents a random value from 1 to 3**

In [3]:
m = folium.Map(location=center, tiles='OpenStreetMap', zoom_start=zoom)

vehicles = []

# VEICOLO B BLU
point_startB = [point_start[0], point_start[1]]
capacityB = 5
vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [capacityB],  # Limite di capacita' del Veicolo    
    profile = 'driving-hgv',
    start = list(reversed(point_startB)), # posizione di partenza veicoli
    end = list(reversed(point_startB)),
))
folium.Marker(point_startB, tooltip=folium.Tooltip(capacityB,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='blue')).add_to(m)

# VEICOLO A ROSSO
point_startA = [point_start[0] + 0.0015, point_start[1] - 0.0015]
capacityA = 5
vehicles.append(ors.optimization.Vehicle(
    id = 1,
    capacity = [capacityA],  # Limite di capacita' del Veicolo    
    profile = 'driving-hgv',
    #restrictions =  ... width,height(in metri), weight(in tons) https://github.com/VROOM-Project/vroom/issues/310
    start = list(reversed(point_startA)), # posizione di partenza veicoli
    end = list(reversed(point_startA)),
))
folium.Marker(point_startA, tooltip=folium.Tooltip(capacityA,permanent=True), icon=folium.Icon(icon='truck', prefix='fa', color='red')).add_to(m)

#ELENCO BIDONI 4 Mezzolombardo e 6 Mezzocorona TOTALE 10 con CONTENUTO CASUALE
job_locs = ccA + ccB

jobs = []
for idx, cc in enumerate(job_locs):
    loc = list(reversed(cc))
    
    amount = random.randrange(0, 3) #random value of bin filling
    
    folium.Marker(loc, tooltip=folium.Tooltip(amount,permanent=True), icon=folium.Icon(icon='archive', prefix='fa', color='green')).add_to(m)
    
    #pass only fill bin
    if amount != 0:
        jobs.append(ors.optimization.Job(
            id = idx,
            location = cc,
            amount = [amount], # Quantita' contenuta nel Bidone
            #priority = [1-100] potrebbe essere utile anche priority
            #ma non e' supportato, bug che verra presto risolto https://github.com/GIScience/openrouteservice-py/issues/39
        ))    
m

In [7]:
clientVrp = ors.Client(base_url=url_vroom, key='')

# USIAMO VROOM ATTRAVERSO LA API PYTHON DI ORS

solution = clientVrp.optimization(
    vehicles = vehicles,
    jobs = jobs,
    geometry = True   #VROOM RESTITUISCE LA GEOMETRIA DEL PERCORSO OLTRE ALLA SOLUZIONE
)

#SOLUZIONE CALCOLATA DA VROOM

solutionA = solution['routes'][0]
solutionB = solution['routes'][1]

folium.PolyLine(
    locations = [list(reversed(coords)) for coords in ors.convert.decode_polyline(solutionA['geometry'])['coordinates']],
    color = 'blue',
    tooltip = folium.Tooltip('Distance: '+str(solutionA['distance']/1000)+'km', permanent=True)
).add_to(m)

folium.PolyLine(
    locations = [list(reversed(coords)) for coords in ors.convert.decode_polyline(solutionB['geometry'])['coordinates']],
    color = 'red',
    tooltip = folium.Tooltip('Distance: '+str(solutionB['distance']/1000)+'km', permanent=True)
).add_to(m)

m

In [90]:
print('Vehicle Red\ntotal distance: '+str(solutionA['distance']/1000)+'km\n')
print('Steps:\n')
for d in solutionA['steps']:
    if d['type']=='job':
        print("bin id: {job} distance: {distance}meters".format(**d))

print('\n')

print('Vehicle Blue\ntotal distance: '+str(solutionB['distance']/1000)+'km\n')
print('Steps:\n')
for d in solutionB['steps']:
    if d['type']=='job':
        print("bin id: {job} distance: {distance}meters".format(**d))

Vehicle Red
total distance: 4.198km

Steps:

bin id: 6 distance: 1444meters
bin id: 7 distance: 1887meters
bin id: 4 distance: 3441meters


Vehicle Blue
total distance: 5.396km

Steps:

bin id: 9 distance: 1479meters
bin id: 2 distance: 3403meters


In [15]:
solution

{'code': 0,
 'summary': {'cost': 1129,
  'unassigned': 0,
  'amount': [8],
  'service': 0,
  'duration': 1129,
  'waiting_time': 0,
  'distance': 9438,
  'computing_times': {'loading': 17, 'solving': 4, 'routing': 27}},
 'unassigned': [],
 'routes': [{'vehicle': 1,
   'cost': 501,
   'amount': [4],
   'service': 0,
   'duration': 501,
   'waiting_time': 0,
   'distance': 5178,
   'steps': [{'type': 'start',
     'location': [11.1066, 46.217],
     'arrival': 0,
     'duration': 0,
     'distance': 0},
    {'type': 'job',
     'location': [11.114301681518556, 46.20900226913922],
     'job': 9,
     'service': 0,
     'waiting_time': 0,
     'arrival': 147,
     'duration': 147,
     'distance': 1208},
    {'type': 'job',
     'location': [11.123356819152832, 46.21419929496722],
     'job': 6,
     'service': 0,
     'waiting_time': 0,
     'arrival': 253,
     'duration': 253,
     'distance': 2423},
    {'type': 'job',
     'location': [11.127262115478516, 46.211972058400264],
     'jo